In [61]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import pickle

In [2]:
training_set = pd.read_csv('..\\processed_data\\training_set.csv')
testing_set = pd.read_csv('..\\processed_data\\testing_set.csv')

In [3]:
X_train = training_set.drop(['CL', 'CD'], axis=1)
y_train = training_set[['CL', 'CD']]

X_test = testing_set.drop(['CL', 'CD'], axis=1)
y_test = testing_set[['CL', 'CD']]

# ---------------------RFR---------------------

In [11]:
rfr_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('rfr', RandomForestRegressor(random_state=42,n_estimators = 100, min_samples_leaf = 1, min_samples_split = 2))
])

In [12]:
rfr_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rfr', RandomForestRegressor(random_state=42))])

In [13]:
preds_rfr = rfr_pipe.predict(X_test)
rfr_train = rfr_pipe.predict(X_train)

mean_absolute_error(y_train, rfr_train)

0.004470728372162766

In [14]:
print('RandomForestRegressor: MAE')
print('-----------------------------------')

print('cl_error : ',mean_absolute_error(y_test['CL'], preds_rfr[:,0]))
print('cd_error : ',mean_absolute_error(y_test['CD'], preds_rfr[:,1]))
print('total_error : ',mean_absolute_error(y_test, preds_rfr))
print('total_error_train_set  : ',mean_absolute_error(y_train, rfr_train))
print('-----------------------------------')


RandomForestRegressor: MAE
-----------------------------------
cl_error :  0.02099930206112297
cd_error :  0.0023511970149253736
total_error :  0.011675249538024166
total_error_train_set  :  0.004470728372162766
-----------------------------------


# ---------------------SVR---------------------

In [10]:
rfr_pipe.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()),
  ('rfr', RandomForestRegressor(random_state=42))],
 'verbose': False,
 'scaler': StandardScaler(),
 'rfr': RandomForestRegressor(random_state=42),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'rfr__bootstrap': True,
 'rfr__ccp_alpha': 0.0,
 'rfr__criterion': 'mse',
 'rfr__max_depth': None,
 'rfr__max_features': 'auto',
 'rfr__max_leaf_nodes': None,
 'rfr__max_samples': None,
 'rfr__min_impurity_decrease': 0.0,
 'rfr__min_impurity_split': None,
 'rfr__min_samples_leaf': 1,
 'rfr__min_samples_split': 2,
 'rfr__min_weight_fraction_leaf': 0.0,
 'rfr__n_estimators': 100,
 'rfr__n_jobs': None,
 'rfr__oob_score': False,
 'rfr__random_state': 42,
 'rfr__verbose': 0,
 'rfr__warm_start': False}

In [15]:
svr = SVR()

svr_pipe = Pipeline(steps = [
    ('scaler', StandardScaler()),
    ('multi_svr', MultiOutputRegressor(svr))
])

In [16]:
svr_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('multi_svr', MultiOutputRegressor(estimator=SVR()))])

In [17]:
preds_svr = svr_pipe.predict(X_test)

print('SVR: MAE')
print('-----------------------------------')

print('cl_error : ',mean_absolute_error(y_test['CL'], preds_svr[:,0]))
print('cd_error : ',mean_absolute_error(y_test['CD'], preds_svr[:,1]))
print('total_error : ',mean_absolute_error(y_test, preds_svr))
print('total_error_train_set  : ',mean_absolute_error(y_train, svr_pipe.predict(X_train)))
print('-----------------------------------')


SVR: MAE
-----------------------------------
cl_error :  0.07729499031815579
cd_error :  0.0677465813788202
total_error :  0.07252078584848799
total_error_train_set  :  0.07236635858761802
-----------------------------------


# ---------------------XGB---------------------

In [58]:
xgb_def = XGBRegressor()

xgb_pipe_def = Pipeline([
    ('scaler', StandardScaler()),
    ('multi_xgb', MultiOutputRegressor(xgb_def))
]
)

In [59]:
xgb_pipe_def.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('multi_xgb',
                 MultiOutputRegressor(estimator=XGBRegressor(base_score=None,
                                                             booster=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_bytree=None,
                                                             enable_categorical=False,
                                                             gamma=None,
                                                             gpu_id=None,
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=None,
                             

In [60]:
preds_xgb_def = xgb_pipe_def.predict(X_test)

print('XGB: MAE')
print('-----------------------------------')

print('cl_error : ',mean_absolute_error(y_test['CL'], preds_xgb_def[:,0]))
print('cd_error : ',mean_absolute_error(y_test['CD'], preds_xgb_def[:,1]))
print('total_error : ',mean_absolute_error(y_test, preds_xgb_def))
print('total_error_train_set  : ',mean_absolute_error(y_train, xgb_pipe_def.predict(X_train)))
print('-----------------------------------')

XGB: MAE
-----------------------------------
cl_error :  0.030474027508028204
cd_error :  0.0022076617634799213
total_error :  0.01634084463575406
total_error_train_set  :  0.012657796831603611
-----------------------------------


# --------------------NN------------------------

In [4]:
from keras.models import Sequential
from keras.layers import Dense

In [12]:
model = Sequential()
model.add(Dense(12, input_dim = 6, activation = 'relu'))
model.add(Dense(6, activation = 'relu'))
model.add(Dense(6, activation = 'relu'))
model.add(Dense(4, activation = 'relu'))
model.add(Dense(2))
model.compile(loss='mae', optimizer='adam')

In [13]:
std = StandardScaler()
train_x_np = np.array(X_train)
train_x_np = std.fit_transform(train_x_np)

train_y_np = np.array(y_train)

test_x_np = np.array(X_test)
test_x_np = std.fit_transform(test_x_np)

test_y_np = np.array(y_test)

In [14]:
model.fit(train_x_np, train_y_np, epochs=200, verbose=1)

Epoch 1/200
313/313 [==============================] - 1s 2ms/step - loss: 0.2852
Epoch 2/200
313/313 [==============================] - 1s 2ms/step - loss: 0.1585
Epoch 3/200
313/313 [==============================] - 1s 2ms/step - loss: 0.0818
Epoch 4/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0636
Epoch 5/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0578
Epoch 6/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0548
Epoch 7/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0533
Epoch 8/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0516
Epoch 9/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0507
Epoch 10/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0503
Epoch 11/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0494
Epoch 12/200
313/313 [==============================] - 0s 1ms/step - loss: 0.0492
Epoch 13/200


In [44]:
mae = model.evaluate(train_x_np, train_y_np)

313/313 [==============================] - 0s 1ms/step - loss: 0.0423


In [46]:
mae_500 = model.evaluate(train_x_np, train_y_np)

313/313 [==============================] - 1s 2ms/step - loss: 0.0347


In [15]:
mae_5layer = model.evaluate(train_x_np, train_y_np)

313/313 [==============================] - 1s 1ms/step - loss: 0.0337


In [18]:
preds = model.predict(test_x_np)

mae_5layer_test = model.evaluate(test_x_np, test_y_np)

88/88 [==============================] - 0s 1ms/step - loss: 0.0446


### From the inital testing of different alogrithms, It seems like Random forest is better and is well enough since the mae is lowest of the batch. But there is some degree of overfitting

### ========================= Hyper parameter tuning ==========================

In [29]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'rfr__n_estimators' : [100, 300, 700, 1000],
    'rfr__max_depth': [2, 5, 8, 10],
    'rfr__min_samples_split' : [2, 5, 10]
}

rf_grid = RandomizedSearchCV(rfr_pipe, parameters, cv = 3)

In [30]:
rf_grid.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('rfr',
                                              RandomForestRegressor(random_state=42))]),
                   param_distributions={'rfr__max_depth': [2, 5, 8, 10],
                                        'rfr__min_samples_split': [2, 5, 10],
                                        'rfr__n_estimators': [100, 300, 700,
                                                              1000]})

In [31]:
rf_grid.best_params_

{'rfr__n_estimators': 1000, 'rfr__min_samples_split': 5, 'rfr__max_depth': 10}

In [33]:
rfr_pipe_tuned = Pipeline(steps = [
    ('scaler', StandardScaler()),
    ('rfr', RandomForestRegressor(n_estimators=1000, min_samples_split=5, max_depth=10, random_state=42))
])

In [34]:
rfr_pipe_tuned.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rfr',
                 RandomForestRegressor(max_depth=10, min_samples_split=5,
                                       n_estimators=1000, random_state=42))])

In [57]:
preds_tuned = rfr_pipe_tuned.predict(X_test)
preds_train = rfr_pipe_tuned.predict(X_train)

print('MAE for tuned train set: ', mean_absolute_error(y_train, preds_train))
print('MAE for tuned test set: ', mean_absolute_error(y_test, preds_tuned))

MAE for tuned train set:  0.01888447433869532
MAE for tuned test set:  0.02236169396921045


In [54]:
rfr_pipe._final_estimator.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

## XGBRegressor is more generalized than Random forest and have less MAE than SVR.

In [48]:
from sklearn.model_selection import RandomizedSearchCV

xgb = XGBRegressor()

xgb_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('multi_xgb', MultiOutputRegressor(xgb))
]
)

params = {
    'multi_xgb__estimator__booster' : ['gbtree', 'gblinear', 'dart'],
    'multi_xgb__estimator__learning_rate': [0.1,0.2,0.3,0.4,0.5],
    'multi_xgb__estimator__max_depth': [1, 2, 3, 4, 5],
    'multi_xgb__estimator__n_estimators' : [100, 200, 300, 400],
    'multi_xgb__estimator__lambda': [0.1, 0.2, 0.5, 0.8]
}

xgb_grid = RandomizedSearchCV(xgb_pipe, params, cv = 5, scoring='neg_mean_absolute_error')

In [49]:
xgb_grid.fit(X_train, y_train)

[21:39:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:39:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:39:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "max_depth" } might not be used.

  This could b

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('multi_xgb',
                                              MultiOutputRegressor(estimator=XGBRegressor(base_score=None,
                                                                                          booster=None,
                                                                                          colsample_bylevel=None,
                                                                                          colsample_bynode=None,
                                                                                          colsample_bytree=None,
                                                                                          enable_categorical=False,
                                                                                          gamma=None,
                                                                        

In [50]:
xgb_grid.best_params_

{'multi_xgb__estimator__n_estimators': 400,
 'multi_xgb__estimator__max_depth': 4,
 'multi_xgb__estimator__learning_rate': 0.4,
 'multi_xgb__estimator__lambda': 0.1,
 'multi_xgb__estimator__booster': 'dart'}

In [51]:
xgb_grid.best_score_

-0.018736307556736557

In [53]:
xgb = XGBRegressor(n_estimators=400, max_depth = 4, learning_rate = 0.4, reg_lambda = 0.1, booster= 'dart')

xgb_pipe = Pipeline(steps = [
    ('scaler', StandardScaler()),
    ('multi_svr', MultiOutputRegressor(xgb))
]
    )

In [54]:
xgb_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('multi_svr',
                 MultiOutputRegressor(estimator=XGBRegressor(base_score=None,
                                                             booster='dart',
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_bytree=None,
                                                             enable_categorical=False,
                                                             gamma=None,
                                                             gpu_id=None,
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=0.4,
                            

In [55]:
preds_xgb = xgb_pipe.predict(X_test)

print('XGB: MAE')
print('-----------------------------------')

print('cl_error : ',mean_absolute_error(y_test['CL'], preds_xgb[:,0]))
print('cd_error : ',mean_absolute_error(y_test['CD'], preds_xgb[:,1]))
print('total_error : ',mean_absolute_error(y_test, preds_xgb))
print('total_error_train_set  : ',mean_absolute_error(y_train, xgb_pipe.predict(X_train)))
print('-----------------------------------')

XGB: MAE
-----------------------------------
cl_error :  0.03173974009600614
cd_error :  0.002732559922681943
total_error :  0.01723615000934404
total_error_train_set  :  0.013695038753838889
-----------------------------------


## Tuned XGBRegressor has a bit high MAE than the default model. Also, it has high no. of trees. So it is optimal to use the default model

In [63]:
filename = '..\\Models\\final_xgb_model.sav'
pickle.dump(xgb_pipe_def, open(filename, 'wb'))